In [18]:
import numpy as np
import random
import keras

# Load prediction files from different classifiers
N_CLF = 4
X_train = np.ndarray((20000, N_CLF))
y_train = np.loadtxt('training_data.txt', skiprows = 1)[:, 0]
X_test = np.loadtxt('test_data.txt', skiprows = 1)
print(X_train.shape, y_train.shape, X_test.shape)

(20000, 3) (20000,) (10000, 1000)


In [15]:
# SVM
X_train[:, 0] = np.loadtxt("submission_svm.csv", skiprows=1)[:, 0]
# Neural net
X_train[:, 1] = np.loadtxt("submission_neural_net.csv", skiprows=1)[:, 0]
# Logistic
X_train[:, 2] = np.loadtxt("submission_logistic.csv", skiprows=1)[:, 0]
# Gradient boost
X_train[:, 3] = np.loadtxt("submission_gdboost.csv", skiprows=1)[:, 0]

In [ ]:
# Logistic regression on predictions
